In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder

In [ ]:
encoder = LabelEncoder()
accelerometer['texte'] = encoder.fit_transform(accelerometer['text'])
# to know which code belong to which label
display(accelerometer['texte'].value_counts())

In [ ]:
accelerometer.dtypes
withoutTS = accelerometer.drop(['time', 'text', 'seconds_elapsed_y', 'seconds_elapsed_x'], axis=1)
withoutTS

In [ ]:
withoutLabelDF = withoutTS.iloc[:, :-1]
withoutLabelDF

In [ ]:
def evaluate(window_size, overlap, feature_sets):
    segments = withoutLabelDF.rolling(window=window_size, center=True, min_periods=window_size).agg(feature_sets)
    segments.columns = ['-'.join(tup).rstrip('-') for tup in segments.columns.values]
    segments = pd.concat([withoutTS.iloc[:, -1], segments], axis=1)
    segments = segments.dropna()
    # segments = segments[::(int)(window_size * overlap)]
    percentage = (int)((window_size)-(window_size * overlap))
    segments = segments[::percentage]
    features_df = segments
    display(features_df)
    # selecting the data and the labels which is the last column
    X = features_df.iloc[:, 1:].values
    y = features_df.iloc[:, :1].values
    #standardization
    sc = StandardScaler()
    X = sc.fit_transform(X)
    #create GNB model
    # classifier = GaussianNB()

    #Create a Gaussian Classifier
    classifier=RandomForestClassifier(n_estimators=100)
    #Train the model using the training sets y_pred=clf.predict(X_test)
    classifier.fit(X,y)
    # y_pred=clf.predict(X_test)

    kfold = StratifiedKFold(n_splits=10)
    # pass your model and KFold object to cross_val_score
    cv_score = cross_val_score(classifier, X, y.ravel(), cv=kfold)
    # Model performance
    print("Accuracy: {:.2f} %".format(cv_score.mean() * 100))
    print("Standard Deviation: {:.2f} %".format(cv_score.std() * 100))
    # y_pred = cross_val_predict(classifier, X, y.ravel(), cv=10)
    # report = classification_report(y, y_pred, target_names=target_names, digits=4, output_dict=True)
    # df = pd.DataFrame(report).transpose()
    # # df.to_csv('C:\\Users\Amer\Desktop\dddd.csv')
    # output_filepath = "C:\\Users\Amer\Desktop\ "
    # df.to_csv(output_filepath + "WS " + str(window_size) + " OL " + str(overlap) + " FS " + str(feature_sets) + ".csv")

In [ ]:
evaluate(50, 0.5, ['min', 'max', 'sum', 'mean', 'std'])